In [6]:
import load_data
import pandas as pd
import torch


In [2]:
transcr_path='paco-cheese/transcr'
data=load_data.load_all_ipus(folder_path=transcr_path,load_words=True)

data.head(2)

,ipu_id,speaker,start_ipu,stop_ipu,text_ipu,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,start_words,stop_words,duration,text_words,request_after_word,turn_after_word,is_ipu_end,dyad
0,0,AA,4.54,4.84,tu as,True,False,True,4.84,True,False,4.54,4.72,0.18,tu,False,False,False,transcr\AAOR
1,0,AA,4.54,4.84,tu as,True,False,True,4.84,True,False,4.72,4.84,0.12,as,False,False,True,transcr\AAOR


In [3]:
data.tail(2)

,ipu_id,speaker,start_ipu,stop_ipu,text_ipu,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,start_words,stop_words,duration,text_words,request_after_word,turn_after_word,is_ipu_end,dyad
110542,681,BN,1122.38,1127.20,deux trois fois d'affilée je te dis c'est bon ...,True,True,False,NaN,False,True,1126.77,1127.20,0.43,trucs,False,False,True,transcr\RPABN
110543,682,RPA,1126.50,1126.79,ouais,False,False,False,NaN,False,False,1126.50,1126.79,0.29,ouais,False,False,True,transcr\RPABN


In [4]:
data[data['turn_after_word']==True]

,ipu_id,speaker,start_ipu,stop_ipu,text_ipu,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,start_words,stop_words,duration,text_words,request_after_word,turn_after_word,is_ipu_end,dyad
378,54,OR,111.12,112.144,c'est le quatre je crois on reprend,True,False,True,111.44,True,False,111.280,111.590,0.310,quatre,True,True,False,transcr\AAOR
429,63,OR,122.72,123.235,et c'est loupé,True,False,True,122.82,True,False,122.790,122.930,0.140,c',True,True,False,transcr\AAOR
485,72,OR,132.96,133.235,mais bon,True,False,True,133.02,True,False,132.960,133.090,0.130,mais,True,True,False,transcr\AAOR
730,108,AA,189.72,190.650,ah ok je sais pas,True,False,True,190.22,True,False,190.040,190.240,0.200,ok,True,True,False,transcr\AAOR
957,137,AA,236.06,236.445,ah bon,True,False,True,236.44,True,False,236.252,236.445,0.193,bon,True,True,True,transcr\AAOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110358,666,BN,1096.50,1096.865,c'est vrai,True,True,True,1097.96,False,True,1096.500,1096.600,0.100,c',True,True,False,transcr\RPABN
110449,674,BN,1110.16,1111.445,du coup tu m'as dit faut juste slider,True,False,True,1111.29,True,False,1110.920,1111.445,0.525,slider,True,True,True,transcr\RPABN
110468,676,RPA,1114.26,1115.458,aller sur la page d'avant et,True,False,True,1115.14,True,False,1114.930,1115.260,0.330,avant,True,True,False,transcr\RPABN
110489,677,BN,1115.14,1119.185,ok psk ouais en pus on tombe pas tous sur les ...,True,True,True,1119.00,True,True,1118.890,1119.185,0.295,aussi,True,True,True,transcr\RPABN


In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification

# df = pd.DataFrame(data)
df=data[0:1000]

# Définir un dataset personnalisé
class TurnDataset(Dataset):
    def __init__(self, tokenizer, df):
        self.tokenizer = tokenizer
        self.texts = df['text_ipu'].tolist()
        self.labels = df['turn_after_word'].astype(int).tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Encodage du texte
        encoded = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoded['input_ids'].flatten(),
            'attention_mask': encoded['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Charger le tokenizer et le modèle
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Créer le dataset et le dataloader
dataset = TurnDataset(tokenizer, df)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Boucle d'entraînement
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(3):  # Parcourir plusieurs fois le dataset
    for i, batch in enumerate(dataloader):
        # Obtenir les entrées
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # Réinitialiser les gradients
        optimizer.zero_grad()

        # forward + backward + optimisation
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print(f"Époque {epoch + 1}, Lot {i + 1}, Perte: {loss.item()}")

# Sauvegarder le modèle
model.save_pretrained('./modele_turn_after_word')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Époque 1, Lot 1, Perte: 0.7821611166000366
Époque 1, Lot 2, Perte: 0.680584192276001
Époque 1, Lot 3, Perte: 0.6281516551971436
Époque 1, Lot 4, Perte: 0.5147973299026489
Époque 1, Lot 5, Perte: 0.48264849185943604
Époque 1, Lot 6, Perte: 0.3882814645767212
Époque 1, Lot 7, Perte: 0.3296492397785187
Époque 1, Lot 8, Perte: 0.23956775665283203
Époque 1, Lot 9, Perte: 0.21700817346572876
Époque 1, Lot 10, Perte: 0.23529016971588135
Époque 1, Lot 11, Perte: 0.19317102432250977
Époque 1, Lot 12, Perte: 0.23023457825183868
Époque 1, Lot 13, Perte: 0.15330082178115845
Époque 1, Lot 14, Perte: 0.1499338448047638
Époque 1, Lot 15, Perte: 0.15243344008922577
Époque 1, Lot 16, Perte: 0.13144996762275696
Époque 1, Lot 17, Perte: 0.12663361430168152
Époque 1, Lot 18, Perte: 0.11807437986135483
Époque 1, Lot 19, Perte: 0.11974818259477615
Époque 1, Lot 20, Perte: 0.13895925879478455
Époque 1, Lot 21, Perte: 0.08609393239021301
Époque 1, Lot 22, Perte: 0.07588842511177063
Époque 1, Lot 23, Perte: 0.


KeyboardInterrupt



In [ ]:
from sklearn.metrics import accuracy_score

# Fonction pour effectuer des prédictions et calculer l'accuracy
def evaluate_model(model, dataloader):
    model.eval()  # Mettre le modèle en mode évaluation
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels'].numpy()

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).numpy()

            predictions.extend(preds)
            true_labels.extend(labels)

    accuracy = accuracy_score(true_labels, predictions)
    return predictions, accuracy

# Entraînement du modèle (le même code que précédemment)

# Après l'entraînement, évaluer le modèle
predictions, accuracy = evaluate_model(model, dataloader)

print("Prédictions:", predictions)
print("Accuracy:", accuracy)
